## ETL Commits Data from Github REST API

From https://github.com/apache/airflow, download commits over the last 6 months.  For this POC, Mar to Aug 2022 is selected for ETL and subsequent analysis.


In [8]:
# to automatically reload jupyter whenever the code in common module is changed
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

from datetime import datetime, timezone
import sys
import time
import requests
import json
import sqlite3
import pandas as pd

import common as comm
import my_credentials


sql_db = 'github_api.db'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### To download commits data

API calls are made in batches i.e., fortnightly batches from Mar to Aug 2022 to address the rate-limit imposed by Github REST API.  Pauses are made between calls to throttle the call frequencies as well.

It takes between 14 to 15mins to collect all responses for the six month periods, using home wifi.

In [9]:
six_mth_period = [('2022-03-01', '2022-03-14'),
                ('2022-03-15', '2022-03-27'),
                ('2022-03-28', '2022-04-09'),
                ('2022-04-10', '2022-04-22'),
                ('2022-04-23', '2022-05-05'),
                ('2022-05-06', '2022-05-18'),
                ('2022-05-19', '2022-05-31'),
                ('2022-06-01', '2022-06-13'),
                ('2022-06-14', '2022-06-26'),
                ('2022-06-27', '2022-07-09'),
                ('2022-07-10', '2022-07-22'),
                ('2022-07-23', '2022-08-05'),
                ('2022-08-06', '2022-08-18'),
                ('2022-08-19', '2022-08-31'),]
                

access_token = my_credentials.secret_token.get('access_token', None)
headers = {'Authorization': access_token}
base_query_url = 'https://api.github.com/search/commits?q=repo:apache/airflow'

response_collected = comm.collect_data(base_query_url, headers, six_mth_period)


### To write data to sqlite3 database

In [10]:
# create table
conn = sqlite3.connect('github_api.db')
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS commits_history
    ([committer_name] TEXT,  
     [committer_email] TEXT,
     [commit_datetime] TEXT,
     [commit_url] TEXT)
""")  
conn.commit()
conn.close()


In [11]:
# delete data b4 insert of data
conn = sqlite3.connect("github_api.db")
cursor = conn.cursor()
query = """
    DELETE FROM commits_history
"""
cursor.execute(query)
conn.commit()


In [12]:
# insert data
from sqlite3 import ProgrammingError


conn = sqlite3.connect("github_api.db")
cursor = conn.cursor()
query = """
    INSERT INTO commits_history
    (committer_name, committer_email, commit_datetime, commit_url)
    VALUES
    (?, ?, ?, ?);    
"""

datapoints = []
for period, list_of_dicts in response_collected.items():
    for dict_obj in list_of_dicts['items']:
        try:
            committer_commit = (dict_obj['commit']['committer']['name'], 
                                dict_obj['commit']['committer']['email'], 
                                dict_obj['commit']['committer']['date'], 
                                dict_obj['commit']['url'],)
            datapoints.append(
                committer_commit
            )
        except ProgrammingError as e:
            print(e)
            continue

      
cursor.executemany(query, tuple(datapoints))  
conn.commit()


In [13]:
# read data
conn = sqlite3.connect("github_api.db")
cursor = conn.cursor()
cursor.execute("""
    SELECT * 
    FROM commits_history
""")



df = pd.DataFrame(cursor.fetchall(), columns=['committer_name', 'committer_email', 'commit_datetime', 'commit_url'])
conn.close()


print(df.head(30))
print(f"df.shape: {df.shape}")

   committer_name     committer_email                commit_datetime  \
0          GitHub  noreply@github.com  2022-03-14T23:17:57.000+01:00   
1          GitHub  noreply@github.com  2022-03-14T14:18:58.000-06:00   
2          GitHub  noreply@github.com  2022-03-14T21:48:32.000+01:00   
3          GitHub  noreply@github.com  2022-03-14T22:18:54.000+01:00   
4          GitHub  noreply@github.com       2022-03-09T12:51:46.000Z   
5          GitHub  noreply@github.com  2022-03-09T19:01:00.000+08:00   
6          GitHub  noreply@github.com  2022-03-09T11:27:05.000+01:00   
7          GitHub  noreply@github.com  2022-03-09T10:45:44.000+02:00   
8          GitHub  noreply@github.com  2022-03-09T10:10:32.000+01:00   
9          GitHub  noreply@github.com  2022-03-09T13:59:52.000+01:00   
10         GitHub  noreply@github.com  2022-03-09T13:15:42.000+01:00   
11         GitHub  noreply@github.com       2022-03-09T12:49:33.000Z   
12         GitHub  noreply@github.com       2022-03-09T14:11:03.